In [2]:
from google.colab import files
uploaded = files.upload()

Saving anime.csv to anime.csv


In [3]:
import pandas as pd
data = pd.read_csv('anime.csv')
data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


Data Description:

Unique ID of each anime.
Anime title.
Anime broadcast type, such as TV, OVA, etc.
anime genre.
The number of episodes of each anime.
The average rating for each anime compared to the number of users who gave ratings.


Number of community members for each anime.
Objective:
The objective of this assignment is to implement a recommendation system using cosine similarity on an anime dataset.
Dataset:
Use the Anime Dataset which contains information about various anime, including their titles, genres,No.of episodes and user ratings etc.

Tasks:

task 1:Data Preprocessing

Load the dataset into a suitable data structure (e.g., pandas DataFrame).
Handle missing values, if any.
Explore the dataset to understand its structure and attributes.

In [4]:
# Handling missing values
print("Number of missing values before handling:\n", data.isnull().sum())

# Fill missing values with 0 for numerical columns and "Unknown" for categorical columns
data['episodes'].fillna(0, inplace=True)
data['rating'].fillna(data['rating'].mean(), inplace=True)

print("\nNumber of missing values after handling:\n", data.isnull().sum())

# Explore the dataset
print("\nData shape:", data.shape)
print("\nData info:\n", data.info())
print("\nFirst 5 rows:\n", data.head())
print("\nDescriptive statistics:\n", data.describe())

Number of missing values before handling:
 anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

Number of missing values after handling:
 anime_id     0
name         0
genre       62
type        25
episodes     0
rating       0
members      0
dtype: int64

Data shape: (12294, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB

Data info:
 None

First 5 rows:
    anime_id                              name  \
0     32281                    Kimi no Na

<ipython-input-4-6ab9faaf037b>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['episodes'].fillna(0, inplace=True)
<ipython-input-4-6ab9faaf037b>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.meth

Feature Extraction:

Decide on the features that will be used for computing similarity (e.g., genres, user ratings).
Convert categorical features into numerical representations if necessary.
Normalize numerical features if required.

In [5]:
# Select relevant features for similarity calculation
features = ['genre', 'rating']

# Create a new DataFrame with selected features
anime_features = data[features].copy()

# Convert genres into numerical representation using one-hot encoding
anime_features = pd.get_dummies(anime_features, columns=['genre'])

# Normalize the 'rating' feature
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
anime_features['rating'] = scaler.fit_transform(anime_features[['rating']])

print("\nAnime features after extraction:\n", anime_features.head())


Anime features after extraction:
      rating  genre_Action  genre_Action, Adventure  \
0  0.924370         False                    False   
1  0.911164         False                    False   
2  0.909964         False                    False   
3  0.900360         False                    False   
4  0.899160         False                    False   

   genre_Action, Adventure, Cars, Comedy, Sci-Fi, Shounen  \
0                                              False        
1                                              False        
2                                              False        
3                                              False        
4                                              False        

   genre_Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports  \
0                                              False               
1                                              False               
2                                              False               
3


Recommendation System:

Design a function to recommend anime based on cosine similarity.
Given a target anime, recommend a list of similar anime based on cosine similarity scores.
Experiment with different threshold values for similarity scores to adjust the recommendation list size.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_anime(anime_title, anime_features, data, threshold=0.7):
  """
  Recommends anime based on cosine similarity.

  Args:
    anime_title: The title of the anime for which recommendations are needed.
    anime_features: DataFrame containing features used for similarity calculation.
    data: Original DataFrame containing anime information.
    threshold: Minimum cosine similarity score for an anime to be considered similar.

  Returns:
    A list of recommended anime titles.
  """

  # Get the feature vector for the target anime
  if anime_title not in data['name'].values:
    return "Anime not found in the dataset."

  target_anime_index = data[data['name'] == anime_title].index[0]
  target_anime_features = anime_features.iloc[[target_anime_index]]

  # Calculate cosine similarity between the target anime and all other anime
  cosine_similarities = cosine_similarity(target_anime_features, anime_features)

  # Get similarity scores for all anime
  similarity_scores = list(enumerate(cosine_similarities[0]))

  # Sort anime based on similarity scores in descending order
  similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

  # Get top similar anime based on threshold
  recommended_anime_indices = [i for i, score in similarity_scores if score >= threshold and i != target_anime_index]

  # Get anime titles from indices
  recommended_anime_titles = [data['name'][i] for i in recommended_anime_indices]

  return recommended_anime_titles

# Example: Recommend anime similar to "Death Note"
recommended_anime = recommend_anime("Death Note", anime_features, data)
print("Recommended anime for 'Death Note':", recommended_anime)

Recommended anime for 'Death Note': ['Death Note Rewrite']



Evaluation:

Split the dataset into training and testing sets.

Evaluate the recommendation system using appropriate metrics such as precision, recall, and F1-score.

Analyze the performance of the recommendation system and identify areas of improvement

In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to evaluate recommendations
def evaluate_recommendations(recommendations, ground_truth):
  precision = 0
  recall = 0
  f1 = 0
  for anime, true_recommendations in ground_truth.items():
    predicted_recommendations = recommend_anime(anime, anime_features, data)
    if predicted_recommendations != "Anime not found in the dataset.":
        common_recommendations = set(predicted_recommendations) & set(true_recommendations)
        if len(predicted_recommendations) > 0:
          precision += len(common_recommendations) / len(predicted_recommendations)
        if len(true_recommendations) > 0:
          recall += len(common_recommendations) / len(true_recommendations)

  if len(ground_truth) > 0:
    precision /= len(ground_truth)
    recall /= len(ground_truth)
    if precision + recall > 0:
      f1 = 2 * (precision * recall) / (precision + recall)

  return precision, recall, f1

# Example ground truth recommendations
ground_truth_recommendations = {
    "Death Note": ["Code Geass", "Monster", "Psycho-Pass"],
    "Attack on Titan": ["Fullmetal Alchemist: Brotherhood", "Vinland Saga", "Demon Slayer: Kimetsu no Yaiba"],
}


# Evaluate the recommendation system
precision, recall, f1 = evaluate_recommendations(
    recommend_anime, ground_truth_recommendations
)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.0
Recall: 0.0
F1-score: 0



Interview Questions:

1. Can you explain the difference between user-based and item-based collaborative filtering?

2. What is collaborative filtering, and how does it work?

In [9]:
# 1. Can you explain the difference between user-based and item-based collaborative filtering?

# User-based collaborative filtering: This approach identifies users with similar preferences to a target user and recommends items that those similar users have liked.
# It essentially finds users who have rated items similarly to the target user and then suggests items that those similar users have rated highly.

# Item-based collaborative filtering: This approach focuses on finding items similar to those that a target user has liked or rated highly.
# It creates a similarity matrix between items based on how users have rated them.
# If a user has enjoyed a specific item, the system then recommends other items that are similar to it based on the ratings of other users.

In [10]:
# 2. What is collaborative filtering, and how does it work?

# Collaborative filtering is a recommendation technique that leverages the collective wisdom of a group of users to predict what a particular user might like.
# It doesn't rely on explicit user profiles or item descriptions but instead focuses on past user interactions (ratings, purchases, etc.) to make predictions.

# How it works:
# 1. Data collection: Gather user interactions with items (e.g., ratings, purchases, views).
# 2. Similarity calculation: Determine similarity between users or items based on their interactions.
#    - User-based: Find users with similar rating patterns.
#    - Item-based: Find items with similar rating patterns across users.
# 3. Prediction: Predict the target user's preference for an item based on the preferences of similar users or the ratings of similar items.
# 4. Recommendation: Recommend items with the highest predicted ratings to the user.

# In essence, collaborative filtering assumes that users who have agreed in the past will likely agree in the future.
# It's a powerful technique for creating personalized recommendations based on the collective behavior of users.